In [1]:
import pandas as pd
import numpy as np
import ut_functions
import sys
sys.path.append("../interface")
import functions
from collections import Counter
% matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ut_functions
import pickle

In [2]:
import importlib
importlib.reload(functions)
from functions import *

In [3]:
def randomSeq(length):
    bases = ['A', 'C', 'G', 'T']
    rand = [np.random.choice(bases) for i in range(length)]
    return ("".join(rand))

In [4]:
def randomReads(seq, length, n):
    seq = seq[0:5000]
    reads = [seq[n:n+length] for n in range(0, len(seq)-(length+1))]
    random_reads = np.random.choice(reads, n)
    return (random_reads)

In [5]:
digestive_names = list(pd.read_csv("../data/matrix2.tsv", sep="\t", index_col=0).columns)[:-1]

In [6]:
m = pd.read_csv('../data/matrix2.tsv', sep="\t", index_col=0)

In [7]:
m = m.fillna(0)

In [8]:
allpaths = set()
viruses = dict()
for ind in m.index.values:
    row = m.loc[ind]
    viruses.setdefault(reverseAnswers()[ind], [])
    for item in row.index.values[:-1]:
        if row[item] != 0:
            viruses[reverseAnswers()[ind]].append(item)
            allpaths.add(item)

NameError: name 'v' is not defined

In [10]:
refs = dict()
for d in digestive_names:
    seq = randomSeq(5000)
    refs[d] = seq

In [11]:
out = open("../data/digest.fasta", "w")
for key, seq in refs.items():
    out.write(">%s\n%s\n" % (key, seq))
out.close()

In [12]:
samplereads = dict()
for k, v in viruses.items():
    print (k)
    all_goodreads = []
    while len(all_goodreads) < 8000:
        for nam in v:
            nreads_small = np.random.choice(np.arange(1000, 2000))
            nreads_med = np.random.choice(np.arange(2000, 5000))
            nreads_high = np.random.choice(np.arange(5000, 8000))
            nreads = np.random.choice([nreads_small, nreads_med, nreads_high])
            goodreads = randomReads(refs[nam.capitalize().replace(" ", "_").replace("_e_", "_E_").replace("_a_", "_A_")], 50, nreads)
            all_goodreads += list(goodreads)
    n_goodreads = np.random.choice(np.arange(6000, 10000))
    allreads = list(np.random.choice(all_goodreads, n_goodreads))
    n_badreads = 10000 - n_goodreads
    n_random_paths = np.random.choice(np.arange(2, 5))
    for i in range(n_random_paths):
        nreads_random_path = np.random.choice(np.arange(1, 25))
        path = np.random.choice(list(set(allpaths) - set(v)))
        seq = refs[path.capitalize().replace(" ", "_").replace("_e_", "_E_").replace("_a_", "_A_")]
        reads = randomReads(seq, 50, nreads_random_path)
        allreads += list(reads)
    n_randomreads = 10000 - len(allreads)
    for i in range(n_randomreads):
        allreads.append("".join([np.random.choice(['A', 'C', 'T', 'G']) for i in range(0, 50)]))
    np.random.shuffle(allreads)
    samplereads[k] = allreads

1
3
2
4
5
6


In [13]:
allpaths = getPathNames()

In [14]:
for key, seqs in samplereads.items():
    out = open("../data/sample_%s_reads_digest.fasta" % key, "w")
    for i, s in enumerate(seqs):
        out.write(">sample_%s_read_%i\n%s\n" % (key, i, s))
    out.close()

In [15]:
importlib.reload(functions)
for dataset in viruses.keys():
    p = functions.mapReadsActual(dataset, typ='digest')
    o = open("../data/%s_digest.pickle" % dataset, "wb")
    pickle.dump(p, o)
    o.close()

In [16]:
for dataset in viruses.keys():
    o = open("../data/%s_digest.pickle" % dataset, "rb")
    r = pickle.load(o)
    o.close()
    rows = []
    for ref, val in r.items():
        rows.append([ref, len(val)])
    tab = pd.DataFrame(rows, columns=['Reference Microbe', "Number of Reads in Sample"])
    tab.to_csv("../data/sample_%s_reads_digest.tsv" % dataset, sep="\t", index=None)